## 題目

Use LSTM & CNN model to classify customized candlestick pattern (at least 3 classes)

### 執行

所有檔案: candlestick_train_cnn.py、candlestick_train_lstm.py

#### 1. Use LSTM model to classify customized candlestick pattern
* candlestick_train_lstm.py

In [9]:
from sklearn.metrics import confusion_matrix
import pickle
import keras
from keras.layers import LSTM
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import Adam


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes):
    x_train = x_train.reshape(-1, n_step, n_input)
    x_test = x_test.reshape(-1, n_step, n_input)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, n_classes)
    y_test = keras.utils.to_categorical(y_test, n_classes)
    return (x_train, x_test, y_train, y_test)

def lstm_model(n_input, n_step, n_hidden, n_classes):
    model = Sequential()
    model.add(LSTM(n_hidden, batch_input_shape=(None, n_step, n_input), unroll=True))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    return model

def train_lstm(model, x_train, y_train, x_test, y_test, 
        learning_rate, training_iters, batch_size):
    adam = Adam(lr=learning_rate)
    model.summary()
    model.compile(optimizer=adam,
        loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train,
        batch_size=batch_size, epochs=training_iters,
        verbose=1, validation_data=(x_test, y_test))

def print_result(data, x_train, x_test, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(x_train)
    test_pred = model.predict_classes(x_test)
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

def candlestick_lstm_main(iters):
    # training parameters
    learning_rate = 0.001
    training_iters = iters
    batch_size = 128

    # model parameters
    n_input = 40
    n_step = 10
    n_hidden = 256
    n_classes = 10

    data = load_pkl('./data/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl')
    x_train, y_train, x_test, y_test = data['train_gaf'], data['train_label'][:, 0], data['test_gaf'], data['test_label'][:, 0]
    x_train, x_test, y_train, y_test = lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes)

    model = lstm_model(n_input, n_step, n_hidden, n_classes)
    train_lstm(model, x_train, y_train, x_test, y_test, learning_rate, 
               training_iters, batch_size)
    scores = model.evaluate(x_test, y_test, verbose=0)
    print('LSTM test accuracy:', scores[1])
    print_result(data, x_train, x_test, model)

In [10]:
candlestick_lstm_main(10)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 256)               304128    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                2570      
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
Total params: 306,698
Trainable params: 306,698
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
118/118 [==============================] - 3s 20ms/step - loss: 2.2044 - accuracy: 0.1951 - val_loss: 1.5961 - val_accuracy: 0.2810
Epoch 2/10
118/118 [==============================] - 2s 18ms/step - loss: 1.5521 - accuracy: 0.3210 - val_loss: 1.5523 - val_accuracy: 0.2882
Epoch 3/10
118/118 [==============================] - 2s 18ms/step - loss: 1.3712 - accuracy

In [11]:
candlestick_lstm_main(20)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 256)               304128    
_________________________________________________________________
dense_8 (Dense)              (None, 10)                2570      
_________________________________________________________________
activation_5 (Activation)    (None, 10)                0         
Total params: 306,698
Trainable params: 306,698
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
118/118 [==============================] - 3s 19ms/step - loss: 2.2063 - accuracy: 0.1975 - val_loss: 1.6521 - val_accuracy: 0.2698
Epoch 2/20
118/118 [==============================] - 2s 18ms/step - loss: 1.6494 - accuracy: 0.2930 - val_loss: 1.5042 - val_accuracy: 0.3286
Epoch 3/20
118/118 [==============================] - 2s 19ms/step - loss: 1.3765 - accuracy

LSTM method with 10 and 20 iterations
* the both final round wouldn't change a lot
* the LSTM test accuracy is 0.717 and 0.8196 respectivily

#### 2. Use CNN model to classify customized candlestick pattern
* candlestick_train_cnn.py

In [12]:
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle

from keras import backend as K
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Activation, MaxPool2D


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def get_cnn_model(params):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(10, 10, 4)))
    model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    return model

def train_model(params, data):
    model = get_cnn_model(params)
    model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])
    hist = model.fit(x=data['train_gaf'], y=data['train_label_arr'],
                     batch_size=params['batch_size'], epochs=params['epochs'], verbose=2)
    return (model, hist)

def print_result(data, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(data['train_gaf'])
    test_pred = model.predict_classes(data['test_gaf'])
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

In [13]:
PARAMS = {}
PARAMS['pkl_name'] = './data/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl'
PARAMS['classes'] = 9
PARAMS['lr'] = 0.01
PARAMS['epochs'] = 10
PARAMS['batch_size'] = 64
PARAMS['optimizer'] = optimizers.SGD(lr=PARAMS['lr'])

# ---------------------------------------------------------
# load data & keras model
data = load_pkl(PARAMS['pkl_name'])
# train cnn model
model, hist = train_model(PARAMS, data)
# train & test result
scores = model.evaluate(data['test_gaf'], data['test_label_arr'], verbose=0)
print('CNN test accuracy:', scores[1])
print_result(data, model)

Epoch 1/10
235/235 - 2s - loss: 1.6311 - accuracy: 0.3929
Epoch 2/10
235/235 - 2s - loss: 0.7968 - accuracy: 0.7135
Epoch 3/10
235/235 - 2s - loss: 0.6037 - accuracy: 0.7801
Epoch 4/10
235/235 - 2s - loss: 0.5258 - accuracy: 0.8085
Epoch 5/10
235/235 - 2s - loss: 0.4833 - accuracy: 0.8236
Epoch 6/10
235/235 - 2s - loss: 0.4611 - accuracy: 0.8313
Epoch 7/10
235/235 - 2s - loss: 0.4373 - accuracy: 0.8425
Epoch 8/10
235/235 - 2s - loss: 0.4182 - accuracy: 0.8464
Epoch 9/10
235/235 - 2s - loss: 0.4098 - accuracy: 0.8493
Epoch 10/10
235/235 - 2s - loss: 0.3944 - accuracy: 0.8561
CNN test accuracy: 0.8525999784469604
[[1785  212  183  113  136  130  105  184  152]
 [   4 1496    0    0    0    0    0    0    0]
 [  16    0 1463    0   21    0    0    0    0]
 [   8   76    0 1292    0    0    0  124    0]
 [   5    0   17    0 1152    0    0    0  326]
 [  36    6    0    2    0 1392    3   61    0]
 [  50    3    4    0    1    0 1368    0   74]
 [   5    8    0  121    0    3    0 1363    

CNN method with 10 iterations
* it performs better than LSTM no matter the iterations are the same, or twice as much
* the CNN test accuracy is .8526

## Reference
-PecuLab Github